In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
from tqdm import tqdm_notebook as tqdmn
import pandas as pd

In [4]:
import os
import time

# loading the path of my driver:
os.environ['PATH'] += r"C:/selenium_drivers"
driver = webdriver.Chrome()

# This is the list where we'll capture the competitors' names and addresses
patents = []

# The name and location of the initial print shop that will go into the search URL

url = 'https://pubmed.ncbi.nlm.nih.gov/?term=artificial+intelligence'

driver.get(url)

WebDriverWait(driver,25).until(EC.visibility_of_element_located((By.CLASS_NAME, "docsum-title")))
time.sleep(1)


patent_title_ = driver.find_element(By.XPATH, '//*[@id="search-results"]/section[1]/div[2]/div/article[1]/div[2]/div[1]/a')

patent_title_.click()

#into first page
# now we gotta itterate thro the pages and extract data
data = []
for _ in range(1000):


    WebDriverWait(driver,25).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="article-details"]')))

    # time.sleep(2)

    title_element = driver.find_element(By.XPATH , '//*[@id="full-view-heading"]/h1')

    patent= {}

    patent['title'] = title_element.text

    patent_authors_div = driver.find_element(By.XPATH , '//*[@id="full-view-heading"]/div[2]/div/div')

    span_elements = driver.find_elements(By.XPATH, '//*[@id="full-view-heading"]/div[2]/div/div/span') #needed for length


    list_of_authors = []
    for i in range(len(span_elements)):
        patent_authors_text = driver.find_element(By.XPATH , f'//*[@id="full-view-heading"]/div[2]/div/div/span[{i+1}]/a').text

        list_of_authors.append(patent_authors_text)

    patent['authors'] = list_of_authors


    # we get the pdf file

    link_to_pdf_element = driver.find_element(By.XPATH, '//*[@id="full-view-identifiers"]/li[2]/span/a') #needed for length
    link_to_pdf = link_to_pdf_element.get_attribute("href")

    try:
        driver2 = webdriver.Chrome()

        driver2.get(link_to_pdf)

        WebDriverWait(driver2,5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/main/aside/div/section[1]')))
        WebDriverWait(driver, 1)
        pdf_file_element = driver2.find_element(By.XPATH , '//*[@id="main-content"]/aside/div/section[1]/ul/li/a')

        pdf_file = pdf_file_element.get_attribute("href")

        patent['pdf_file'] = pdf_file

        driver2.get(pdf_file)
    except:
        driver2.quit()
        patent['pdf_file'] = link_to_pdf

    finally:
        driver2.quit()



    try : 

        abstract_element = driver.find_element(By.XPATH, '//*[@id="eng-abstract"]/p') 
        abstract = abstract_element.text

        patent["abstract"] = abstract
    except:
        patent["abstract"] = None 

    # key_words_element = driver.find_element(By.XPATH, '//*[@id="abstract"]') 
    # key_words = key_words_element.text
    # patent['key_words'] = key_words.split("Keywords: ")[1]

    next_page = driver.find_element(By.XPATH, f'//*[@id="adjacent-navigation"]/div[{2 if _ == 0 else 3}]/a') 
    next_page.click()

    data.append(patent)




NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF665ACAD02+56930]
	(No symbol) [0x00007FF665A3F602]
	(No symbol) [0x00007FF6658F42E5]
	(No symbol) [0x00007FF6658D1D4C]
	(No symbol) [0x00007FF6659623F7]
	(No symbol) [0x00007FF665977891]
	(No symbol) [0x00007FF66595BA43]
	(No symbol) [0x00007FF66592D438]
	(No symbol) [0x00007FF66592E4D1]
	GetHandleVerifier [0x00007FF665E46F8D+3711213]
	GetHandleVerifier [0x00007FF665EA04CD+4077101]
	GetHandleVerifier [0x00007FF665E9865F+4044735]
	GetHandleVerifier [0x00007FF665B69736+706710]
	(No symbol) [0x00007FF665A4B8DF]
	(No symbol) [0x00007FF665A46AC4]
	(No symbol) [0x00007FF665A46C1C]
	(No symbol) [0x00007FF665A368D4]
	BaseThreadInitThunk [0x00007FF94D487344+20]
	RtlUserThreadStart [0x00007FF94E3A26B1+33]


In [12]:
data

[{'title': 'Artificial intelligence to deep learning: machine intelligence approach for drug discovery',
  'authors': ['Rohan Gupta',
   'Devesh Srivastava',
   'Mehar Sahu',
   'Swati Tiwari',
   'Rashmi K Ambasta',
   'Pravir Kumar'],
  'pdf_file': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8040371/pdf/11030_2021_Article_10217.pdf',
  'abstract': 'Drug designing and development is an important area of research for pharmaceutical companies and chemical scientists. However, low efficacy, off-target delivery, time consumption, and high cost impose a hurdle and challenges that impact drug design and discovery. Further, complex and big data from genomics, proteomics, microarray data, and clinical trials also impose an obstacle in the drug discovery pipeline. Artificial intelligence and machine learning technology play a crucial role in drug discovery and development. In other words, artificial neural networks and deep learning algorithms have modernized the area. Machine learning and d

In [13]:
DF =pd.DataFrame(data)

In [14]:
DF

,title,authors,pdf_file,abstract
0,Artificial intelligence to deep learning: mach...,"[Rohan Gupta, Devesh Srivastava, Mehar Sahu, S...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,Drug designing and development is an important...
1,Artificial intelligence in medicine,"[A N Ramesh, C Kambhampati, J R T Monson, P J ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Introduction: Artificial intelligence is a bra...
2,Artificial Intelligence in Dentistry-Narrative...,"[Agata Ossowska, Aida Kusiak, Dariusz Świetlik]",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,"Nowadays, artificial intelligence (AI) is beco..."
3,Artificial Intelligence in Anesthesiology: Cur...,"[Daniel A Hashimoto, Elan Witkowski, Lei Gao, ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Artificial intelligence has been advancing in ...
4,Artificial intelligence in medicine,"[Pavel Hamet, Johanne Tremblay]",https://doi.org/10.1016/j.metabol.2017.01.011,Artificial Intelligence (AI) is a general term...
5,Introduction to artificial intelligence in med...,"[Yoav Mintz, Ronit Brodie]",https://doi.org/10.1080/13645706.2019.1575882,The term Artificial Intelligence (AI) was coin...
6,Artificial Intelligence and Ophthalmology,"[Kadircan Keskinbora, Fatih Güven]",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Artificial intelligence is advancing rapidly a...
7,Artificial intelligence in cardiology,[Dipti Itchhaporia],https://doi.org/10.1016/j.tcm.2020.11.007,This review examines the current state and app...
8,Artificial intelligence in healthcare: An esse...,"[Mei Chen, Michel Decary]",https://doi.org/10.1177/0840470419873123,Artificial Intelligence (AI) is evolving rapid...
9,Artificial Intelligence in Nuclear Medicine,"[Felix Nensa, Aydin Demircioglu, Christoph Ris...",https://doi.org/10.2967/jnumed.118.220590,Despite the great media attention for artifici...
